<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/potentiality_ai_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import random
import os

# 1. Synthetic Dataset
class PotentialityDataset(Dataset):
    """
    Creates synthetic data (X, Y) where
      Y1 = sin(x0 + x1)
      Y2 = cos(x2 - x3)
      Y3 = Y1 + Y2    <-- physics constraint
    """
    def __init__(self, num_samples=1000, input_dim=6, seed=42):
        super().__init__()
        random.seed(seed)
        np.random.seed(seed)
        self.X = np.random.uniform(-2, 2, size=(num_samples, input_dim)).astype(np.float32)
        y1 = np.sin(self.X[:, 0] + self.X[:, 1])
        y2 = np.cos(self.X[:, 2] - self.X[:, 3])
        y3 = y1 + y2
        self.Y = np.stack([y1, y2, y3], axis=1).astype(np.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx]), torch.from_numpy(self.Y[idx])

# 2. Model Definition
class PotentialityAI(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, p_dropout=0.1):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(p_dropout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.drop(x)
        return self.fc2(x)

# 3. Physics-informed residual
def physics_residual(y_pred):
    # Enforces y1 + y2 == y3
    return y_pred[:, 0] + y_pred[:, 1] - y_pred[:, 2]

# 4. Training & Evaluation Utilities
def train_one_epoch(model, loader, optimizer, mse_loss, lambda_phy, device):
    model.train()
    total_loss = 0.0
    for X_batch, Y_batch in loader:
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        optimizer.zero_grad()
        preds = model(X_batch)
        mse = mse_loss(preds, Y_batch)
        phy = torch.mean(physics_residual(preds)**2)
        loss = mse + lambda_phy * phy
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * X_batch.size(0)
    return total_loss / len(loader.dataset)

def evaluate(model, loader, mse_loss, lambda_phy, device):
    model.eval()
    total_loss = total_mse = total_phy = 0.0
    with torch.no_grad():
        for X_batch, Y_batch in loader:
            X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
            preds = model(X_batch)
            mse = mse_loss(preds, Y_batch)
            phy = torch.mean(physics_residual(preds)**2)
            loss = mse + lambda_phy * phy
            total_loss += loss.item() * X_batch.size(0)
            total_mse  += mse.item()  * X_batch.size(0)
            total_phy  += phy.item()  * X_batch.size(0)
    n = len(loader.dataset)
    return total_loss/n, total_mse/n, total_phy/n

def mc_dropout_predict(model, X, T=50):
    """Return mean and std over T stochastic forward passes."""
    model.train()  # keep dropout on
    preds = []
    for _ in range(T):
        preds.append(model(X).unsqueeze(0))
    preds = torch.cat(preds, dim=0)  # shape [T, batch, out_dim]
    return preds.mean(dim=0), preds.std(dim=0)

# 5. Main Training Loop
def main():
    # Hyperparameters
    input_dim, hidden_dim, output_dim = 6, 64, 3
    batch_size, epochs = 64, 200
    lr, lambda_phy = 1e-3, 1.0
    patience = 20
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Prepare data
    full_dataset = PotentialityDataset(num_samples=2000, input_dim=input_dim)
    train_size = int(0.8 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_ds, val_ds = torch.utils.data.random_split(full_dataset, [train_size, val_size])
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(val_ds, batch_size=batch_size)

    # Model, loss, optimizer, scheduler
    model     = PotentialityAI(input_dim, hidden_dim, output_dim, p_dropout=0.1).to(device)
    mse_loss  = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)

    # Training with early stopping
    best_val = float("inf")
    wait     = 0
    history  = {"train_loss": [], "val_loss": []}

    for ep in range(1, epochs+1):
        train_loss = train_one_epoch(model, train_loader, optimizer, mse_loss, lambda_phy, device)
        val_loss, _, _ = evaluate(model, val_loader, mse_loss, lambda_phy, device)
        scheduler.step(val_loss)

        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        print(f"Epoch {ep:03d} | Train {train_loss:.4f} | Val {val_loss:.4f}")

        # Checkpoint & early stop
        if val_loss < best_val:
            best_val = val_loss
            torch.save(model.state_dict(), "best_potentiality_ai.pth")
            wait = 0
        else:
            wait += 1
            if wait > patience:
                print("Early stopping.")
                break

    # Load best model
    model.load_state_dict(torch.load("best_potentiality_ai.pth"))

    # 6. Visualization
    # Loss curves
    plt.figure()
    plt.plot(history["train_loss"], label="Train Loss")
    plt.plot(history["val_loss"],   label="Val Loss")
    plt.yscale("log")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.title("Training & Validation Loss")
    plt.savefig("loss_curve.png", dpi=150)

    # Parity plot on one validation batch
    model.eval()
    X_val, Y_val = next(iter(val_loader))
    X_val, Y_val = X_val.to(device), Y_val.to(device)
    preds = model(X_val)
    plt.figure(figsize=(6,6))
    plt.scatter(Y_val.cpu().numpy().ravel(), preds.detach().cpu().numpy().ravel(), alpha=0.5)
    mn, mx = float(min(Y_val.min(), preds.min())), float(max(Y_val.max(), preds.max()))
    plt.plot([mn, mx], [mn, mx], 'r--')
    plt.xlabel("True")
    plt.ylabel("Pred")
    plt.title("Parity Plot")
    plt.savefig("parity.png", dpi=150)

    # Uncertainty histogram for Y1
    mean_pred, std_pred = mc_dropout_predict(model, X_val, T=50)
    u = std_pred[:, 0].detach().cpu().numpy()
    plt.figure()
    plt.hist(u, bins=30, color='orange')
    plt.xlabel("Std Dev of Y1")
    plt.title("Uncertainty Distribution (Y1)")
    plt.savefig("uncertainty_hist.png", dpi=150)

    print("Training complete. Plots saved: loss_curve.png, parity.png, uncertainty_hist.png")

if __name__ == "__main__":
    main()